In [1]:
from datetime import datetime
import datetime as dt
import time
from subgrounds.subgraph import SyntheticField, FieldPath
from subgrounds.subgrounds import Subgrounds
import pandas as pd
import duckdb as db
import seaborn as sns
import matplotlib.pyplot as plt
import IPython as ip
import  os as os


****************************************************************************************************************
****************************************************************************************************************
AS ALWAYS START WITH SILO
https://app.silo.finance/silo/0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db?screen=deposit

In [2]:
daysLimit = int(input('How many days back of history to pulll?'))

In [3]:
sg = Subgrounds()
apiKey = input('API Key, please')

In [6]:
#for when we save the files
file = input('Selet a folder to save output') ##enter your file path here - the file is in the repo "summary_stats.csv".
file = file+'/data'
if not os.path.exists(file):
    os.makedirs(file)

In [4]:
#SILO subgraph: https://gateway.thegraph.com/api/[api-key]/subgraphs/id/6rcT2GzBXGoi3ZMus92LHajM2b9qWGj7Z3dDhPxoUogB
silo = sg.load_api('https://gateway.thegraph.com/api/'+apiKey+'/subgraphs/id/6rcT2GzBXGoi3ZMus92LHajM2b9qWGj7Z3dDhPxoUogB')

JSONDecodeError: [Errno Expecting value] : 0

In [5]:
oldBorrowsDb = pd.DataFrame()
siloFileName = file+'/siloFinanceOhmBorrows.csv'
try:
    oldBorrowsDb = pd.read_csv (siloFileName)
except:
    oldBorrowsDb = pd.DataFrame()
oldBorrowsDb.head(100)

NameError: name 'file' is not defined

In [7]:
maxSiloBorrowTimestamp = 0
try:
    maxSiloBorrowTimestamp = (max(0,oldBorrowsDb['transaction_timestamp'].max()))-5
except:
        maxSiloBorrowTimestamp = 0
maxSiloBorrowTimestamp = int(maxSiloBorrowTimestamp)
maxSiloBorrowTimestamp

0

In [8]:
#0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db

In [9]:
##this captures the ENTIRE list of people who borrowed OHM
borrowTicker = 0
borrowslist = pd.DataFrame()
siloBorrowsDb = pd.DataFrame()
borrowListLength = 1000
datediff = 0
now=0
daysAgo=0
#daysLimit = 45
exit = False
while exit==False:
    skipValue = (borrowTicker)*(1000)
    borrows = silo.Query.borrows(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db'
                ,'timestamp_gte': maxSiloBorrowTimestamp
                } #gOHM market
    )

    borrowslist = sg.query_df([
        borrows.id
        ,borrows.hash
        ,borrows.blockNumber
        ,borrows.timestamp
        ,borrows.account.id
        ,borrows.market.id
        ,borrows.market.protocol.id
        ,borrows.market.rates.rate
        ,borrows.market.rates.duration
        ,borrows.market.rates.side
        ,borrows.market.rates.type
        ,borrows.market.rates.token.id
        ,borrows.asset.name
        ,borrows.asset.symbol
        ,borrows.asset.decimals
        ,borrows.asset.lastPriceUSD
        ,borrows.amount
        ,borrows.amountUSD
    ])

    siloBorrowsDb=pd.concat([siloBorrowsDb, borrowslist])
    recordTimestamp1 = siloBorrowsDb.iat[borrowTicker,3]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    siloBorrowsDbLength = len(siloBorrowsDb)
    borrowListLength = len(borrowslist)
    recordID = siloBorrowsDb.iat[borrowTicker,0]
    borrow = siloBorrowsDb.iat[borrowTicker,1]
    print("iterations: ", borrowTicker, "Lines skipped: ",skipValue, "records collected: ", borrowListLength, " - latest recordID: ",borrow, " ", recordID , " - borrows DB length: ", siloBorrowsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", borrowTicker, "records collected: ", borrowListLength, " - userdb length: ", borrowsDbLength)
    borrowTicker = borrowTicker+1

    if daysAgo>daysLimit: exit =True
    if borrowListLength<1000: exit =True

siloBorrowsDb.columns = siloBorrowsDb.columns.str.replace("borrows", "transaction")
siloBorrowsDb['event']='borrow'
siloBorrowsDb.drop_duplicates
print("Done. Total Records collected: ", siloBorrowsDbLength)

iterations:  0 Lines skipped:  0 records collected:  30  - latest recordID:  0x3063cfbced7d9431862437e31adcc3581e9df1cd87d35e641422b5c6693b060f   0x3063cfbced7d9431862437e31adcc3581e9df1cd87d35e641422b5c6693b060f-111  - borrows DB length:  30  - latest record from:  2023-02-20 19:26:47 3 days ago
Done. Total Records collected:  30


In [10]:
siloBorrowsDb.head(100)

,transaction_id,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_market_protocol_id,transaction_market_rates_rate,transaction_market_rates_duration,transaction_market_rates_side,transaction_market_rates_type,transaction_market_rates_token_id,transaction_asset_name,transaction_asset_symbol,transaction_asset_decimals,transaction_asset_lastPriceUSD,transaction_amount,transaction_amountUSD,event
0,0x3063cfbced7d9431862437e31adcc3581e9df1cd87d3...,0x3063cfbced7d9431862437e31adcc3581e9df1cd87d3...,16674201,1676950007,0xf659faac0401cc19110321dcc7989c101cced8e4,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,5.656345,None,BORROWER,VARIABLE,0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5,Olympus,OHM,9,10.379649,6420000000000,69617.631237,borrow
1,0x3063cfbced7d9431862437e31adcc3581e9df1cd87d3...,0x3063cfbced7d9431862437e31adcc3581e9df1cd87d3...,16674201,1676950007,0xf659faac0401cc19110321dcc7989c101cced8e4,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,VARIABLE,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Olympus,OHM,9,10.379649,6420000000000,69617.631237,borrow
2,0x3063cfbced7d9431862437e31adcc3581e9df1cd87d3...,0x3063cfbced7d9431862437e31adcc3581e9df1cd87d3...,16674201,1676950007,0xf659faac0401cc19110321dcc7989c101cced8e4,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,VARIABLE,0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac,Olympus,OHM,9,10.379649,6420000000000,69617.631237,borrow
3,0x3063cfbced7d9431862437e31adcc3581e9df1cd87d3...,0x3063cfbced7d9431862437e31adcc3581e9df1cd87d3...,16674201,1676950007,0xf659faac0401cc19110321dcc7989c101cced8e4,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,2.285402,None,LENDER,VARIABLE,0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5,Olympus,OHM,9,10.379649,6420000000000,69617.631237,borrow
4,0x3063cfbced7d9431862437e31adcc3581e9df1cd87d3...,0x3063cfbced7d9431862437e31adcc3581e9df1cd87d3...,16674201,1676950007,0xf659faac0401cc19110321dcc7989c101cced8e4,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,LENDER,VARIABLE,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Olympus,OHM,9,10.379649,6420000000000,69617.631237,borrow
5,0x3063cfbced7d9431862437e31adcc3581e9df1cd87d3...,0x3063cfbced7d9431862437e31adcc3581e9df1cd87d3...,16674201,1676950007,0xf659faac0401cc19110321dcc7989c101cced8e4,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,LENDER,VARIABLE,0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac,Olympus,OHM,9,10.379649,6420000000000,69617.631237,borrow
6,0x264ecb6cc9089ebf8fcb6f8ae5ce26358b15b0551e61...,0x264ecb6cc9089ebf8fcb6f8ae5ce26358b15b0551e61...,16648400,1676636519,0xcb6e1613029d790c00f89296808f278d6dc25b2f,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,5.656345,None,BORROWER,VARIABLE,0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5,Olympus,OHM,9,10.379649,58000000000,615.628222,borrow
7,0x264ecb6cc9089ebf8fcb6f8ae5ce26358b15b0551e61...,0x264ecb6cc9089ebf8fcb6f8ae5ce26358b15b0551e61...,16648400,1676636519,0xcb6e1613029d790c00f89296808f278d6dc25b2f,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,VARIABLE,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Olympus,OHM,9,10.379649,58000000000,615.628222,borrow
8,0x264ecb6cc9089ebf8fcb6f8ae5ce26358b15b0551e61...,0x264ecb6cc9089ebf8fcb6f8ae5ce26358b15b0551e61...,16648400,1676636519,0xcb6e1613029d790c00f89296808f278d6dc25b2f,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,VARIABLE,0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac,Olympus,OHM,9,10.379649,58000000000,615.628222,borrow
9,0x264ecb6cc9089ebf8fcb6f8ae5ce26358b15b0551e61...,0x264ecb6cc9089ebf8fcb6f8ae5ce26358b15b0551e

In [11]:
siloBorrowsDb['transactionReadableAmount'] = siloBorrowsDb['transaction_amount']/(10**(siloBorrowsDb['transaction_asset_decimals']))
siloBorrowsDb['transactionReadableAmount']=siloBorrowsDb['transactionReadableAmount'].astype('float')
siloBorrowsDb['dateTime'] =(pd.to_datetime(siloBorrowsDb['transaction_timestamp'],unit='s'))



#we only want borrowing in GOHM from those that borrowed, so drop anyone that is not on that side
#we only want withdrawals in GOHM from those that borrowed, so drop anyone that is not on that side
siloBorrowsDb.drop(siloBorrowsDb.loc[(siloBorrowsDb['transaction_market_rates_side']!='BORROWER') &(siloBorrowsDb['transaction_asset_symbol']!='OHM') ].index, inplace=True)
#siloBorrowsDb.drop(siloWithdrawsDb.loc[(siloBorrowsDb['transaction_asset_name']=='Olympus')].index, inplace=True)
siloBorrowsDb = db.query("select distinct * from siloBorrowsDb where lower(transaction_asset_symbol) = 'ohm'").df()

siloBorrowEventsDbTimedClean = siloBorrowsDb[['event','transaction_hash','transaction_blockNumber','transaction_timestamp','transaction_account_id','transaction_market_id','transaction_asset_name','transaction_asset_symbol', 'transactionReadableAmount','transaction_asset_lastPriceUSD','transaction_amountUSD','dateTime']].copy()
siloBorrowEventsDbTimedClean.sort_values(by=['event','transaction_hash','transaction_blockNumber','transaction_timestamp','transaction_account_id','transaction_market_id','transaction_asset_name','transaction_asset_symbol', 'transactionReadableAmount','transaction_asset_lastPriceUSD','transaction_amountUSD','dateTime'])
siloBorrowEventsDbTimedClean.drop_duplicates(subset=['transaction_hash'],keep='first', inplace=True)
siloBorrowEventsDbTimedClean['calDate'] = siloBorrowEventsDbTimedClean['dateTime'].dt.normalize()
siloBorrowEventsDbTimedClean.head(100)

,event,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_asset_name,transaction_asset_symbol,transactionReadableAmount,transaction_asset_lastPriceUSD,transaction_amountUSD,dateTime,calDate
0,borrow,0x3063cfbced7d9431862437e31adcc3581e9df1cd87d3...,16674201,1676950007,0xf659faac0401cc19110321dcc7989c101cced8e4,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,Olympus,OHM,6420.000000,10.379649,69617.631237,2023-02-21 03:26:47,2023-02-21
6,borrow,0x264ecb6cc9089ebf8fcb6f8ae5ce26358b15b0551e61...,16648400,1676636519,0xcb6e1613029d790c00f89296808f278d6dc25b2f,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,Olympus,OHM,58.000000,10.379649,615.628222,2023-02-17 12:21:59,2023-02-17
12,borrow,0x6726ea9fa581ab0a5fa26b8a112c95e690500f446c39...,16628130,1676391467,0x9b8b04b6f82cd5e1dae58ca3614d445f93defc5c,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,Olympus,OHM,1480.000000,10.379649,15782.938441,2023-02-14 16:17:47,2023-02-14
18,borrow,0x5fca09aaf64619a5d644ed392a64f5bd08b9e6b4b46e...,16627404,1676382743,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,Olympus,OHM,175.320428,10.379649,1812.167469,2023-02-14 13:52:23,2023-02-14


In [12]:
path =file+'/siloFinanceGohmBorrows.csv'
siloBorrowEventsDbTimedClean.to_csv(path, index = False)

path =file+'/siloFinanceGohmBorrows_piped.csv'
siloBorrowEventsDbTimedClean.to_csv(path, index = False, sep='|')
print(path, 'file saved,', len(siloBorrowEventsDbTimedClean),' records')

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\ohmBorrowing/data/siloFinanceGohmBorrows_piped.csv file saved, 4  records


In [13]:
totalBorrowedOHM = sum(siloBorrowEventsDbTimedClean['transactionReadableAmount'])
totalBorrowedUSD = sum(siloBorrowEventsDbTimedClean['transaction_amountUSD'])
print('A total of ',totalBorrowedOHM, 'OHM has been borrowed, or USD$',round(totalBorrowedUSD,2),  'over time')

A total of  8133.320427647 OHM has been borrowed, or USD$ 87828.37 over time


************************************************************************************
*************************************************************************************
SILO DEPOSITS

In [14]:
oldSiloDepositsDb = pd.DataFrame()
siloFileName = file+'/siloFinanceGohmDeposits.csv'
try:
    oldSiloDepositsDb = pd.read_csv (siloFileName)
except:
    oldSiloDepositsDb = pd.DataFrame()
oldSiloDepositsDb.head(100)

,event,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_asset_name,transaction_asset_symbol,transactionReadableAmount,transaction_asset_lastPriceUSD,transaction_amountUSD,dateTime,calDate
0,deposit,0x1c37394aaabc2445a66b61a360cf08ae096b7582e2ed...,16646360,1676611691,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,Olympus,OHM,10.000000,10.463267,105.925340,2023-02-17 05:28:11,2023-02-17
1,deposit,0xf9bbcc923182fb6406e97fce0f92c22c87a284d55812...,16627144,1676379599,0x245cc372c84b3645bf0ffe6538620b04a217988b,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,Olympus,OHM,20000.000000,10.463267,206726.334541,2023-02-14 12:59:59,2023-02-14
2,deposit,0x0aa351eb8ff1ca2bd6c4e65229f83c7e5b596d72c889...,16625541,1676360171,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,Olympus,OHM,2.514698,10.463267,26.317329,2023-02-14 07:36:11,2023-02-14


In [15]:
maxSiloDepositsTimestamp = 0
try:
    maxSiloDepositsTimestamp = (max(0,oldSiloDepositssDb['transaction_timestamp'].max()))-5
except:
        maxSiloDepositsTimestamp = 0
maxSiloDepositsTimestamp = int(maxSiloDepositsTimestamp)
maxSiloDepositsTimestamp

0

In [16]:
##this captures the ENTIRE list of people who deposited OHM
depositTicker = 0
depositslist = pd.DataFrame()
siloDepositsDb = pd.DataFrame()
depositListLength = 1000
datediff = 0
now=0
daysAgo=0
#daysLimit = 45
exit = False
while exit==False:
    skipValue = (depositTicker)*(1000)
    deposits = silo.Query.deposits(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db'
                ,'timestamp_gte': maxSiloDepositsTimestamp
                } #gOHM market
    )

    depositslist = sg.query_df([
        deposits.id
        ,deposits.hash
        ,deposits.blockNumber
        ,deposits.timestamp
        ,deposits.account.id
        ,deposits.market.id
        ,deposits.market.protocol.id
        ,deposits.market.rates.rate
        ,deposits.market.rates.duration
        ,deposits.market.rates.side
        ,deposits.market.rates.type
        ,deposits.market.rates.token.id
        ,deposits.asset.name
        ,deposits.asset.symbol
        ,deposits.asset.decimals
        ,deposits.asset.lastPriceUSD
        ,deposits.amount
        ,deposits.amountUSD
    ])

    siloDepositsDb=pd.concat([siloDepositsDb, depositslist])
    recordTimestamp1 = siloDepositsDb.iat[depositTicker,3]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    siloDepositsDbLength = len(siloDepositsDb)
    siloDepositListLength = len(depositslist)
    recordID = siloDepositsDb.iat[depositTicker,0]
    deposit = siloDepositsDb.iat[depositTicker,1]
    print("iterations: ", depositTicker, "Lines skipped: ",skipValue, "records collected: ", siloDepositListLength, " - latest recordID: ",deposit, " ", recordID , " - deposits DB length: ", siloDepositsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", depositTicker, "records collected: ", depositListLength, " - userdb length: ", depositsDbLength)
    depositTicker = depositTicker+1

    if daysAgo>daysLimit: exit =True
    if siloDepositListLength<1000: exit =True

siloDepositsDb.columns = siloDepositsDb.columns.str.replace("deposits", "transaction")
siloDepositsDb['event']='deposit'
siloDepositsDb.drop_duplicates
print("Done. Total Records collected: ", siloDepositsDbLength)

iterations:  0 Lines skipped:  0 records collected:  72  - latest recordID:  0x0ceec06c6de27fa4e52ce6dd1cc7af9421d0a70a75833671a38d263469aa157a   0x0ceec06c6de27fa4e52ce6dd1cc7af9421d0a70a75833671a38d263469aa157a-218  - deposits DB length:  72  - latest record from:  2023-02-20 19:25:23 3 days ago
Done. Total Records collected:  72


In [17]:
siloDepositsDb.head(10)

,transaction_id,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_market_protocol_id,transaction_market_rates_rate,transaction_market_rates_duration,transaction_market_rates_side,transaction_market_rates_type,transaction_market_rates_token_id,transaction_asset_name,transaction_asset_symbol,transaction_asset_decimals,transaction_asset_lastPriceUSD,transaction_amount,transaction_amountUSD,event
0,0x0ceec06c6de27fa4e52ce6dd1cc7af9421d0a70a7583...,0x0ceec06c6de27fa4e52ce6dd1cc7af9421d0a70a7583...,16674194,1676949923,0xf659faac0401cc19110321dcc7989c101cced8e4,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,5.656345,None,BORROWER,VARIABLE,0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5,XAI Stablecoin,XAI,18,1.005117,99941732127782654852940,100524.640608,deposit
1,0x0ceec06c6de27fa4e52ce6dd1cc7af9421d0a70a7583...,0x0ceec06c6de27fa4e52ce6dd1cc7af9421d0a70a7583...,16674194,1676949923,0xf659faac0401cc19110321dcc7989c101cced8e4,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,VARIABLE,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,XAI Stablecoin,XAI,18,1.005117,99941732127782654852940,100524.640608,deposit
2,0x0ceec06c6de27fa4e52ce6dd1cc7af9421d0a70a7583...,0x0ceec06c6de27fa4e52ce6dd1cc7af9421d0a70a7583...,16674194,1676949923,0xf659faac0401cc19110321dcc7989c101cced8e4,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,VARIABLE,0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac,XAI Stablecoin,XAI,18,1.005117,99941732127782654852940,100524.640608,deposit
3,0x0ceec06c6de27fa4e52ce6dd1cc7af9421d0a70a7583...,0x0ceec06c6de27fa4e52ce6dd1cc7af9421d0a70a7583...,16674194,1676949923,0xf659faac0401cc19110321dcc7989c101cced8e4,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,2.285402,None,LENDER,VARIABLE,0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5,XAI Stablecoin,XAI,18,1.005117,99941732127782654852940,100524.640608,deposit
4,0x0ceec06c6de27fa4e52ce6dd1cc7af9421d0a70a7583...,0x0ceec06c6de27fa4e52ce6dd1cc7af9421d0a70a7583...,16674194,1676949923,0xf659faac0401cc19110321dcc7989c101cced8e4,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,LENDER,VARIABLE,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,XAI Stablecoin,XAI,18,1.005117,99941732127782654852940,100524.640608,deposit
5,0x0ceec06c6de27fa4e52ce6dd1cc7af9421d0a70a7583...,0x0ceec06c6de27fa4e52ce6dd1cc7af9421d0a70a7583...,16674194,1676949923,0xf659faac0401cc19110321dcc7989c101cced8e4,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,LENDER,VARIABLE,0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac,XAI Stablecoin,XAI,18,1.005117,99941732127782654852940,100524.640608,deposit
6,0xa3c1b116f0bdff039f4969bd23a7fb83b8ddb2f5efac...,0xa3c1b116f0bdff039f4969bd23a7fb83b8ddb2f5efac...,16648394,1676636447,0xcb6e1613029d790c00f89296808f278d6dc25b2f,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,5.656345,None,BORROWER,VARIABLE,0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5,Wrapped Ether,WETH,18,1655.794128,500000000000000000,829.050000,deposit
7,0xa3c1b116f0bdff039f4969bd23a7fb83b8ddb2f5efac...,0xa3c1b116f0bdff039f4969bd23a7fb83b8ddb2f5efac...,16648394,1676636447,0xcb6e1613029d790c00f89296808f278d6dc25b2f,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,VARIABLE,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,1655.794128,500000000000000000,829.050000,deposit
8,0xa3c1b116f0bdff039f4969bd23a7fb83b8ddb2f5efac...,0xa3c1b116f0bdff039f4969bd23a7fb83b8ddb2f5efac...,16648394,1676636447,0xcb6e1613029d790c00f89296808f278d6dc25b2f,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,VARIABLE,0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac,

In [18]:
siloDepositsDb['transactionReadableAmount'] = siloDepositsDb['transaction_amount']/(10**(siloDepositsDb['transaction_asset_decimals']))
siloDepositsDb['transactionReadableAmount']=siloDepositsDb['transactionReadableAmount'].astype('float')
siloDepositsDb['dateTime'] =(pd.to_datetime(siloDepositsDb['transaction_timestamp'],unit='s'))
siloDepositsDb.head(100)

#we only want deposits in GOHM from those that borrowed, so drop anyone that is not on that side
siloDepositsDb.drop(siloDepositsDb.loc[siloDepositsDb['transaction_market_rates_side']!='BORROWER'].index, inplace=True)
siloDepositsDb.drop(siloDepositsDb.loc[siloDepositsDb['transaction_asset_symbol']!='OHM'].index, inplace=True)


siloDepositEventsDbTimedClean = siloDepositsDb[['event','transaction_hash','transaction_blockNumber','transaction_timestamp','transaction_account_id','transaction_market_id','transaction_asset_name','transaction_asset_symbol', 'transactionReadableAmount','transaction_asset_lastPriceUSD','transaction_amountUSD','dateTime']].copy()
siloDepositEventsDbTimedClean.sort_values(by=['event','transaction_hash','transaction_blockNumber','transaction_timestamp','transaction_account_id','transaction_market_id','transaction_asset_name','transaction_asset_symbol', 'transactionReadableAmount','transaction_asset_lastPriceUSD','transaction_amountUSD','dateTime'])
siloDepositEventsDbTimedClean.drop_duplicates(subset=['transaction_hash'],keep='first', inplace=True)
siloDepositEventsDbTimedClean['calDate'] = siloDepositEventsDbTimedClean['dateTime'].dt.normalize()
siloDepositEventsDbTimedClean.head(100)

,event,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_asset_name,transaction_asset_symbol,transactionReadableAmount,transaction_asset_lastPriceUSD,transaction_amountUSD,dateTime,calDate
12,deposit,0x1c37394aaabc2445a66b61a360cf08ae096b7582e2ed...,16646360,1676611691,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,Olympus,OHM,10.000000,10.379649,105.925340,2023-02-17 05:28:11,2023-02-17
42,deposit,0xf9bbcc923182fb6406e97fce0f92c22c87a284d55812...,16627144,1676379599,0x245cc372c84b3645bf0ffe6538620b04a217988b,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,Olympus,OHM,20000.000000,10.379649,206726.334541,2023-02-14 12:59:59,2023-02-14
48,deposit,0x0aa351eb8ff1ca2bd6c4e65229f83c7e5b596d72c889...,16625541,1676360171,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,Olympus,OHM,2.514698,10.379649,26.317329,2023-02-14 07:36:11,2023-02-14


In [19]:
path =file+'/siloFinanceGohmDeposits.csv'
siloDepositEventsDbTimedClean.to_csv(path, index = False)

path =file+'/siloFinanceGohmDeposits_piped.csv'
siloDepositEventsDbTimedClean.to_csv(path, index = False, sep='|')
print(path, 'file saved,', len(siloDepositEventsDbTimedClean),' records')

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\ohmBorrowing/data/siloFinanceGohmDeposits_piped.csv file saved, 3  records


In [20]:
totalDeposited = sum(siloDepositEventsDbTimedClean['transactionReadableAmount'])
print('There has been a total of',round(totalDeposited,2),  'OHM deposited cumulatively as collateral  to date')

There has been a total of 20012.51 OHM deposited cumulatively as collateral  to date


In [21]:
totalDepositedUSD = sum(siloDepositEventsDbTimedClean['transaction_amountUSD'])
print('That is a value of USD$',round(totalDepositedUSD,2))

That is a value of USD$ 206858.58


********************************************************************
********************************************************************
Now we need withdrawals to counter the deposits and see current state

In [22]:
oldSiloWithdrawsDb = pd.DataFrame()
siloFileName = file+'/siloFinanceGohmWithdraws.csv'
try:
    oldSiloWithdrawsDb = pd.read_csv (siloFileName)
except:
    oldSiloWithdrawsDb = pd.DataFrame()
oldSiloWithdrawsDb.head(100)

,event,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_asset_name,transaction_asset_symbol,transactionReadableAmount,transaction_asset_lastPriceUSD,transaction_amountUSD,dateTime,calDate
0,withdraw,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,16627397,1676382659,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,Olympus,OHM,2.514698,10.463267,25.992718,2023-02-14 13:50:59,2023-02-14


In [23]:
maxSiloWithdrawsTimestamp = 0
try:
    maxSiloWithdrawsTimestamp = (max(0,oldSiloWithdrawssDb['transaction_timestamp'].max()))-5
except:
        maxSiloWithdrawsTimestamp = 0
maxSiloWithdrawsTimestamp = int(maxSiloWithdrawsTimestamp)
maxSiloWithdrawsTimestamp

0

In [24]:
##this captures the ENTIRE list of people who withdrawed OHM
withdrawTicker = 0
withdrawslist = pd.DataFrame()
siloWithdrawsDb = pd.DataFrame()
withdrawListLength = 1000
datediff = 0
now=0
daysAgo=0
#daysLimit = 45
exit = False
while exit==False:
    skipValue = (withdrawTicker)*(1000)
    withdraws = silo.Query.withdraws(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db'
                ,'timestamp_gte': maxSiloWithdrawsTimestamp
                ,'asset':'0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5'
                ,'amount_gte': 0
                } #gOHM market
    )

    withdrawslist = sg.query_df([
        withdraws.id
        ,withdraws.hash
        ,withdraws.blockNumber
        ,withdraws.timestamp
        ,withdraws.account.id
        ,withdraws.market.id
        ,withdraws.market.protocol.id
        ,withdraws.market.rates.rate
        ,withdraws.market.rates.duration
        ,withdraws.market.rates.side
        ,withdraws.market.rates.type
        ,withdraws.market.rates.token.id
        ,withdraws.asset.name
        ,withdraws.asset.symbol
        ,withdraws.asset.decimals
        ,withdraws.asset.lastPriceUSD
        ,withdraws.amount
        ,withdraws.amountUSD
    ])

    siloWithdrawsDb=pd.concat([siloWithdrawsDb, withdrawslist])
    recordTimestamp1 = siloWithdrawsDb.iat[withdrawTicker,3]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1)
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    silowithdrawsDbLength = len(siloWithdrawsDb)
    silowithdrawListLength = len(withdrawslist)
    recordID = siloWithdrawsDb.iat[withdrawTicker,0]
    withdraw = siloWithdrawsDb.iat[withdrawTicker,1]
    print("iterations: ", withdrawTicker, "Lines skipped: ",skipValue, "records collected: ", silowithdrawListLength, " - latest recordID: ",withdraw, " ", recordID , " - withdraws DB length: ", silowithdrawsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", withdrawTicker, "records collected: ", withdrawListLength, " - userdb length: ", withdrawsDbLength)
    withdrawTicker = withdrawTicker+1

    if daysAgo>daysLimit: exit =True
    if silowithdrawListLength<1000: exit =True

siloWithdrawsDb.columns = siloWithdrawsDb.columns.str.replace("withdraws", "transaction")
siloWithdrawsDb['event']='withdraw'
siloWithdrawsDb.drop_duplicates
print("Done. Total Records collected: ", silowithdrawsDbLength)

iterations:  0 Lines skipped:  0 records collected:  6  - latest recordID:  0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7f23b9e1bb32b1fac6ec0   0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7f23b9e1bb32b1fac6ec0-259  - withdraws DB length:  6  - latest record from:  2023-02-14 05:50:59 9 days ago
Done. Total Records collected:  6


In [25]:
siloWithdrawEventsDbTimedClean = pd.DataFrame()
siloWithdrawsDb['transactionReadableAmount'] = siloWithdrawsDb['transaction_amount']/(10**(siloWithdrawsDb['transaction_asset_decimals']))
siloWithdrawsDb['transactionReadableAmount']=siloWithdrawsDb['transactionReadableAmount'].astype('float')
siloWithdrawsDb['dateTime'] =(pd.to_datetime(siloWithdrawsDb['transaction_timestamp'],unit='s'))

#we only want withdrawals in GOHM from those that borrowed, so drop anyone that is not on that side
siloWithdrawsDb.drop(siloWithdrawsDb.loc[siloWithdrawsDb['transaction_market_rates_side']!='BORROWER'].index, inplace=True)
siloWithdrawsDb.drop(siloWithdrawsDb.loc[siloWithdrawsDb['transaction_asset_symbol']!='OHM'].index, inplace=True)

siloWithdrawEventsDbTimedClean = siloWithdrawsDb[['event','transaction_hash','transaction_blockNumber','transaction_timestamp','transaction_account_id','transaction_market_id','transaction_asset_name','transaction_asset_symbol', 'transactionReadableAmount','transaction_asset_lastPriceUSD','transaction_amountUSD','dateTime']].copy()
siloWithdrawEventsDbTimedClean.sort_values(by=['event','transaction_hash','transaction_blockNumber','transaction_timestamp','transaction_account_id','transaction_market_id','transaction_asset_name','transaction_asset_symbol', 'transactionReadableAmount','transaction_asset_lastPriceUSD','transaction_amountUSD','dateTime'])
siloWithdrawEventsDbTimedClean.drop_duplicates(subset=['transaction_hash'],keep='first', inplace=True)
siloWithdrawEventsDbTimedClean['calDate'] = siloWithdrawEventsDbTimedClean['dateTime'].dt.normalize()
siloWithdrawsDb.head(100)

,transaction_id,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_market_protocol_id,transaction_market_rates_rate,transaction_market_rates_duration,transaction_market_rates_side,...,transaction_market_rates_token_id,transaction_asset_name,transaction_asset_symbol,transaction_asset_decimals,transaction_asset_lastPriceUSD,transaction_amount,transaction_amountUSD,event,transactionReadableAmount,dateTime
0,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,16627397,1676382659,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,5.656345,None,BORROWER,...,0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5,Olympus,OHM,9,10.379649,2514698283,25.992718,withdraw,2.514698,2023-02-14 13:50:59
1,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,16627397,1676382659,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Olympus,OHM,9,10.379649,2514698283,25.992718,withdraw,2.514698,2023-02-14 13:50:59
2,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,16627397,1676382659,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,...,0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac,Olympus,OHM,9,10.379649,2514698283,25.992718,withdraw,2.514698,2023-02-14 13:50:59


In [26]:
path =file+'/siloFinanceGohmWithdraws.csv'
siloWithdrawEventsDbTimedClean.to_csv(path, index = False)

path =file+'/siloFinanceGohmWithdraws_piped.csv'
siloWithdrawEventsDbTimedClean.to_csv(path, index = False, sep='|')
print(path, 'file saved,', len(siloWithdrawEventsDbTimedClean),' records')

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\ohmBorrowing/data/siloFinanceGohmWithdraws_piped.csv file saved, 1  records


In [27]:
totalWithdrawed = sum(siloWithdrawEventsDbTimedClean['transactionReadableAmount'])
print('There has been a total of',round(totalWithdrawed,2),  'OHM withdrawn cumulatively as collateral  to date')
totalWithdrawedUSD =sum(siloWithdrawEventsDbTimedClean['transaction_amountUSD'])
print('That is a value of USD$',round(totalWithdrawedUSD,2))

There has been a total of 2.51 OHM withdrawn cumulatively as collateral  to date
That is a value of USD$ 25.99


In [28]:
collateralBalance = totalDeposited-totalWithdrawed
print(totalDeposited, ' - ',totalWithdrawed,'=', collateralBalance, 'currently deposited as collateral in Silo' )

20012.514698283  -  2.514698283 = 20010.0 currently deposited as collateral in Silo


*************************************************************
*************************************************************
LET'S LOOK AT THE EULER MARKET
https://app.euler.finance/market/0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5
https://gateway.thegraph.com/api/[api-key]/subgraphs/id/8cLf29KxAedWLVaEqjV8qKomdwwXQxjptBZFrqWNH5u2

STARTING WITH BORROWS (hasn't launched yet as of 2/16/23)

In [29]:
euler = sg.load_api('https://gateway.thegraph.com/api/'+apiKey+'/subgraphs/id/8cLf29KxAedWLVaEqjV8qKomdwwXQxjptBZFrqWNH5u2')

In [30]:
oldEulerBorrowsDb = pd.DataFrame()
EulerBorrowsFileName = file+'/eulerFinanceOhmBorrows.csv'
try:
    oldEulerBorrowsDb = pd.read_csv (EulerBorrowsFileName)
except:
    oldEulerBorrowsDb = pd.DataFrame()
oldEulerBorrowsDb.head(100)

""


In [31]:
maxEulerBorrowTimestamp = 0
try:
    maxEulerBorrowTimestamp = (max(0,oldBorrowsDb['transaction_timestamp'].max()))-5
except:
        maxEulerBorrowTimestamp = 0
maxEulerBorrowTimestamp = int(maxEulerBorrowTimestamp)
maxEulerBorrowTimestamp

0

##SAMPLE QUERY
eulerBorrowsDb = pd.DataFrame()
deposits = euler.Query.deposits(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ #'timestamp_gte': maxEulerdepositTimestamp
                #,'hash': '0x301069fd24dd70d15cdcbebd4f076efe486a82f347240703adcb1713a6582e98'
                'market':'0x8a6622d293d5187a39f8dba7f6f1ae7fa527483b'
                ,'asset':'0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5'
                }
    )

depositslist = sg.query_df([
    deposits.id
    ,deposits.hash
    ,deposits.logIndex
    ,deposits.timestamp
    ,deposits.to
    ,deposits.asset.id
    ,deposits.asset.symbol
    ,deposits.asset.decimals
    ,deposits.asset.lastPriceUSD
    ,deposits.asset.lastPriceBlockNumber
    ,deposits.amount
    ,deposits.amountUSD
    ,deposits._select("from")
])

depositslist.head(10)

In [72]:
##this captures the ENTIRE list of people who borrowed OHM
borrowTicker = 0
borrowslist = pd.DataFrame()
eulerBorrowsDb = pd.DataFrame()
eulerBorrowsDbLength = 1000
datediff = 0
now=0
daysAgo=0
#daysLimit = 45
exit = False

try:
    while exit==False:
        skipValue = (borrowTicker)*(1000)
        borrows = euler.Query.borrows(
                orderBy='timestamp',
                orderDirection='desc',
                first=1000,
                skip = skipValue,
                where={ #'timestamp_gte': maxEulerBorrowTimestamp
                        #,'hash': '0x301069fd24dd70d15cdcbebd4f076efe486a82f347240703adcb1713a6582e98'
                        'market':'0x8a6622d293d5187a39f8dba7f6f1ae7fa527483b'
                       # ,'asset':'0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5'
                        }
            )

        borrowslist = sg.query_df([
            borrows.id
            ,borrows.hash
            ,borrows.logIndex
            ,borrows.timestamp
            ,borrows.to
            ,borrows.asset.id
            ,borrows.asset.symbol
            ,borrows.asset.decimals
            ,borrows.asset.lastPriceUSD
            ,borrows.asset.lastPriceBlockNumber
            ,borrows.amount
            ,borrows.amountUSD
            ,borrows._select("from")
        ])

        eulerBorrowsDb=pd.concat([eulerBorrowsDb, borrowslist])

        recordTimestamp1 = eulerBorrowsDb.iat[borrowTicker,3]
        recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
        now = (int(dt.datetime.utcnow().timestamp()))
        datediff=abs(int(now) - recordTimestamp1)
        daysAgo = int(datediff/86400)
        eulerBorrowsDbLength = len(eulerBorrowsDb)
        eulerBorrowListLength = len(borrowslist)
        recordID = eulerBorrowsDb.iat[borrowTicker,0]
        borrow = eulerBorrowsDb.iat[borrowTicker,1]
        print("iterations: ", borrowTicker, "Lines skipped: ",skipValue, "records collected: ", eulerBorrowListLength, " - latest recordID: ",borrow, " ", recordID , " - borrows DB length: ", eulerBorrowsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
        #print("iterations: ", borrowTicker, "records collected: ", borrowListLength, " - userdb length: ", borrowsDbLength)
        borrowTicker = borrowTicker+1

        if daysAgo>daysLimit: exit =True
        if eulerBorrowListLength<1000: exit =True

    eulerBorrowsDb.columns = eulerBorrowsDb.columns.str.replace("borrows", "transaction")
    eulerBorrowsDb['event']='borrow'
    eulerBorrowsDb.drop_duplicates
    print("Done. Total Records collected: ", eulerBorrowsDbLength)
except:
    print('Error.No evemts in DB, perhaps?')

Error.No evemts in DB, perhaps?


In [33]:
eulerBorrowsDb.head(10)

""


****************************************************************************************************************
****************************************************************************************************************
LET'S LOOK AT THE EULER DEPOSIT EVENTS

In [34]:
oldEulerDepositsDb = pd.DataFrame()
EulerDepositsFileName = file+'/eulerFinanceOhmDeposits.csv'
try:
    oldEulerDepositsDb = pd.read_csv (EulerDepositsFileName)
except:
    oldEulerDepositsDb = pd.DataFrame()
oldEulerDepositsDb.head(100)

,transaction_id,transaction_hash,transaction_logIndex,transaction_timestamp,transaction_to,transaction_asset_id,transaction_asset_symbol,transaction_asset_decimals,transaction_asset_lastPriceUSD,transaction_asset_lastPriceBlockNumber,transaction_amount,transaction_amountUSD,transaction_from,event,transactionReadableAmount,dateTime
0,0x301069fd24dd70d15cdcbebd4f076efe486a82f34724...,0x301069fd24dd70d15cdcbebd4f076efe486a82f34724...,199,1676615723,0x8a6622d293d5187a39f8dba7f6f1ae7fa527483b,0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5,OHM,9,10.463267,16678974,4973203839,52.678831,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,deposit,4.973204,2023-02-17 06:35:23
1,0xa7495eba745bd67279969c1b8687f816e0d83a60bf0c...,0xa7495eba745bd67279969c1b8687f816e0d83a60bf0c...,123,1676379695,0x8a6622d293d5187a39f8dba7f6f1ae7fa527483b,0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5,OHM,9,10.463267,16678974,30000000000000,310089.501811,0x245cc372c84b3645bf0ffe6538620b04a217988b,deposit,30000.000000,2023-02-14 13:01:35


In [35]:
maxEulerDepositTimestamp = 0
try:
    maxEulerDepositTimestamp = (max(0,oldEulerDepositsDb['deposit_timestamp'].max()))-5
except:
        maxEulerDepositTimestamp = 0
maxEulerDepositTimestamp = int(maxEulerDepositTimestamp)
maxEulerDepositTimestamp

0

In [36]:
##this captures the ENTIRE list of people who Deposited OHM
depositTicker = 0
depositlist = pd.DataFrame()
eulerDepositsDb = pd.DataFrame()
eulerDepositsDbLength = 1000
datediff = 0
now=0
daysAgo=0
#daysLimit = 45
exit = False
while exit==False:
    skipValue = (depositTicker)*(1000)
    deposits = euler.Query.deposits(
            orderBy='timestamp',
            orderDirection='desc',
            first=1000,
            skip = skipValue,
            where={ #'timestamp_gte': maxEulerDepositTimestamp
                    #,'hash': '0x301069fd24dd70d15cdcbebd4f076efe486a82f347240703adcb1713a6582e98'
                    'market':'0x8a6622d293d5187a39f8dba7f6f1ae7fa527483b'
                    ,'asset':'0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5'
                    }
        )

    depositslist = sg.query_df([
        deposits.id
        ,deposits.hash
        ,deposits.logIndex
        ,deposits.timestamp
        ,deposits.to
        ,deposits.asset.id
        ,deposits.asset.symbol
        ,deposits.asset.decimals
        ,deposits.asset.lastPriceUSD
        ,deposits.asset.lastPriceBlockNumber
        ,deposits.amount
        ,deposits.amountUSD
        ,deposits._select("from")
    ])

    eulerDepositsDb=pd.concat([eulerDepositsDb, depositslist])

    recordTimestamp1 = eulerDepositsDb.iat[depositTicker,3]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    eulerDepositsDbLength = len(eulerDepositsDb)
    eulerDepositListLength = len(depositslist)
    recordID = eulerDepositsDb.iat[depositTicker,0]
    deposit = eulerDepositsDb.iat[depositTicker,1]
    print("iterations: ", depositTicker, "Lines skipped: ",skipValue, "records collected: ", eulerDepositListLength, " - latest recordID: ",deposit, " ", recordID , " - deposits DB length: ", eulerDepositsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", depositTicker, "records collected: ", depositListLength, " - userdb length: ", depositsDbLength)
    depositTicker = depositTicker+1

    if daysAgo>daysLimit: exit =True
    if eulerDepositListLength<1000: exit =True

eulerDepositsDb.columns = eulerDepositsDb.columns.str.replace("deposits", "transaction")
eulerDepositsDb['event']='deposit'
eulerDepositsDb.drop_duplicates
print("Done. Total Records collected: ", eulerDepositsDbLength)

iterations:  0 Lines skipped:  0 records collected:  3  - latest recordID:  0xffd4f953670dec6f8ae7d0d79391572912b056b99f6c1bb6bc98a4ecfd90130e   0xffd4f953670dec6f8ae7d0d79391572912b056b99f6c1bb6bc98a4ecfd90130e-265  - deposits DB length:  3  - latest record from:  2023-02-23 00:00:23 0 days ago
Done. Total Records collected:  3


In [37]:
eulerDepositsDb.head(10)

,transaction_id,transaction_hash,transaction_logIndex,transaction_timestamp,transaction_to,transaction_asset_id,transaction_asset_symbol,transaction_asset_decimals,transaction_asset_lastPriceUSD,transaction_asset_lastPriceBlockNumber,transaction_amount,transaction_amountUSD,transaction_from,event
0,0xffd4f953670dec6f8ae7d0d79391572912b056b99f6c...,0xffd4f953670dec6f8ae7d0d79391572912b056b99f6c...,265,1677139223,0x8a6622d293d5187a39f8dba7f6f1ae7fa527483b,0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5,OHM,9,10.379649,16694105,450000000000,4781.601428,0x642c8ed496a3d05a0ed08911c01f652a0c4b84d4,deposit
1,0x301069fd24dd70d15cdcbebd4f076efe486a82f34724...,0x301069fd24dd70d15cdcbebd4f076efe486a82f34724...,199,1676615723,0x8a6622d293d5187a39f8dba7f6f1ae7fa527483b,0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5,OHM,9,10.379649,16694105,4973203839,52.678831,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,deposit
2,0xa7495eba745bd67279969c1b8687f816e0d83a60bf0c...,0xa7495eba745bd67279969c1b8687f816e0d83a60bf0c...,123,1676379695,0x8a6622d293d5187a39f8dba7f6f1ae7fa527483b,0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5,OHM,9,10.379649,16694105,30000000000000,310089.501811,0x245cc372c84b3645bf0ffe6538620b04a217988b,deposit


In [38]:
eulerdepositsDbTimedClean = pd.DataFrame()
eulerDepositsDb['transactionReadableAmount'] = eulerDepositsDb['transaction_amount']/(10**(eulerDepositsDb['transaction_asset_decimals']))

eulerDepositsDb['transactionReadableAmount']=eulerDepositsDb['transactionReadableAmount'].astype('float')
eulerDepositsDb['dateTime'] =(pd.to_datetime(eulerDepositsDb['transaction_timestamp'],unit='s'))
eulerDepositsDb.head(10)

,transaction_id,transaction_hash,transaction_logIndex,transaction_timestamp,transaction_to,transaction_asset_id,transaction_asset_symbol,transaction_asset_decimals,transaction_asset_lastPriceUSD,transaction_asset_lastPriceBlockNumber,transaction_amount,transaction_amountUSD,transaction_from,event,transactionReadableAmount,dateTime
0,0xffd4f953670dec6f8ae7d0d79391572912b056b99f6c...,0xffd4f953670dec6f8ae7d0d79391572912b056b99f6c...,265,1677139223,0x8a6622d293d5187a39f8dba7f6f1ae7fa527483b,0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5,OHM,9,10.379649,16694105,450000000000,4781.601428,0x642c8ed496a3d05a0ed08911c01f652a0c4b84d4,deposit,450.000000,2023-02-23 08:00:23
1,0x301069fd24dd70d15cdcbebd4f076efe486a82f34724...,0x301069fd24dd70d15cdcbebd4f076efe486a82f34724...,199,1676615723,0x8a6622d293d5187a39f8dba7f6f1ae7fa527483b,0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5,OHM,9,10.379649,16694105,4973203839,52.678831,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,deposit,4.973204,2023-02-17 06:35:23
2,0xa7495eba745bd67279969c1b8687f816e0d83a60bf0c...,0xa7495eba745bd67279969c1b8687f816e0d83a60bf0c...,123,1676379695,0x8a6622d293d5187a39f8dba7f6f1ae7fa527483b,0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5,OHM,9,10.379649,16694105,30000000000000,310089.501811,0x245cc372c84b3645bf0ffe6538620b04a217988b,deposit,30000.000000,2023-02-14 13:01:35


In [39]:
eulerDepositsDb['transactionReadableAmount'] = eulerDepositsDb['transaction_amount']*1e-9
eulerDepositsDb['dateTime'] =(pd.to_datetime(eulerDepositsDb['transaction_timestamp'],unit='s'))
eulerDepositsDb.head(10)

,transaction_id,transaction_hash,transaction_logIndex,transaction_timestamp,transaction_to,transaction_asset_id,transaction_asset_symbol,transaction_asset_decimals,transaction_asset_lastPriceUSD,transaction_asset_lastPriceBlockNumber,transaction_amount,transaction_amountUSD,transaction_from,event,transactionReadableAmount,dateTime
0,0xffd4f953670dec6f8ae7d0d79391572912b056b99f6c...,0xffd4f953670dec6f8ae7d0d79391572912b056b99f6c...,265,1677139223,0x8a6622d293d5187a39f8dba7f6f1ae7fa527483b,0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5,OHM,9,10.379649,16694105,450000000000,4781.601428,0x642c8ed496a3d05a0ed08911c01f652a0c4b84d4,deposit,450.000000,2023-02-23 08:00:23
1,0x301069fd24dd70d15cdcbebd4f076efe486a82f34724...,0x301069fd24dd70d15cdcbebd4f076efe486a82f34724...,199,1676615723,0x8a6622d293d5187a39f8dba7f6f1ae7fa527483b,0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5,OHM,9,10.379649,16694105,4973203839,52.678831,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,deposit,4.973204,2023-02-17 06:35:23
2,0xa7495eba745bd67279969c1b8687f816e0d83a60bf0c...,0xa7495eba745bd67279969c1b8687f816e0d83a60bf0c...,123,1676379695,0x8a6622d293d5187a39f8dba7f6f1ae7fa527483b,0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5,OHM,9,10.379649,16694105,30000000000000,310089.501811,0x245cc372c84b3645bf0ffe6538620b04a217988b,deposit,30000.000000,2023-02-14 13:01:35


In [40]:
eulerDepositsDb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 16 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   transaction_id                          3 non-null      object        
 1   transaction_hash                        3 non-null      object        
 2   transaction_logIndex                    3 non-null      int64         
 3   transaction_timestamp                   3 non-null      int64         
 4   transaction_to                          3 non-null      object        
 5   transaction_asset_id                    3 non-null      object        
 6   transaction_asset_symbol                3 non-null      object        
 7   transaction_asset_decimals              3 non-null      int64         
 8   transaction_asset_lastPriceUSD          3 non-null      float64       
 9   transaction_asset_lastPriceBlockNumber  3 non-null      in

In [41]:
totalDepositedEuler = sum(eulerDepositsDb['transactionReadableAmount'])
print('There has been a total of', round(totalDepositedEuler, 2), 'OHM deposited cumulatively to date')

There has been a total of 30454.97 OHM deposited cumulatively to date


In [42]:
totalDepositedUSDEuler = sum(eulerDepositsDb['transaction_amountUSD'])
print('That\'s $', round(totalDepositedUSDEuler, 2), 'USD deposited at last price')

That's $ 314923.78 USD deposited at last price


In [43]:
path =file+'/eulerFinanceOhmDeposits.csv'
eulerDepositsDb.to_csv(path, index = False)

path =file+'/eulerFinanceOhmDeposits_piped.csv'
eulerDepositsDb.to_csv(path, index = False, sep='|')
print(path, 'file saved,', len(siloWithdrawEventsDbTimedClean),' records')

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\ohmBorrowing/data/eulerFinanceOhmDeposits_piped.csv file saved, 1  records


****************************************************************************************************************
****************************************************************************************************************
EULER WITHDRAWAL EVENTS (none as of 02/19)

In [44]:
oldEulerWithdrawsDb = pd.DataFrame()
EulerFileName = file+'/EulerFinanceGohmWithdraws.csv'
try:
    oldEulerWithdrawsDb = pd.read_csv (EulerFileName)
except:
    oldEulerWithdrawsDb = pd.DataFrame()
oldEulerWithdrawsDb.head(100)

""


In [45]:
maxEulerWithdrawsTimestamp = 0
try:
    maxEulerWithdrawsTimestamp = (max(0,oldEulerWithdrawssDb['transaction_timestamp'].max()))-5
except:
        maxEulerWithdrawsTimestamp = 0
maxEulerWithdrawsTimestamp = int(maxEulerWithdrawsTimestamp)
maxEulerWithdrawsTimestamp

0

In [46]:
##this captures the ENTIRE list of people who withdrawed OHM
try:
    withdrawTicker = 0
    withdrawslist = pd.DataFrame()
    eulerWithdrawsDb = pd.DataFrame()
    withdrawListLength = 1000
    datediff = 0
    now=0
    daysAgo=0
    #daysLimit = 45
    exit = False
    while exit==False:
        skipValue = (withdrawTicker)*(1000)
        withdraws = euler.Query.withdraws(
            orderBy='timestamp',
            orderDirection='desc',
            first=1000,
            skip = skipValue,
            where={ #'timestamp_gte': maxEulerBorrowTimestamp
                    #,'hash': '0x301069fd24dd70d15cdcbebd4f076efe486a82f347240703adcb1713a6582e98'
                    'market':'0x8a6622d293d5187a39f8dba7f6f1ae7fa527483b'
                    ,'asset':'0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5'
                    }
            )

        withdrawslist = sg.query_df([
            withdraws.id
            ,withdraws.hash
            ,withdraws.logIndex
            ,withdraws.timestamp
            ,withdraws.to
            ,withdraws.asset.id
            ,withdraws.asset.symbol
            ,withdraws.asset.decimals
            ,withdraws.asset.lastPriceUSD
            ,withdraws.asset.lastPriceBlockNumber
            ,withdraws.amount
            ,withdraws.amountUSD
            ,withdraws._select("from")
        ])

        eulerWithdrawsDb=pd.concat([eulerWithdrawsDb, withdrawslist])
        recordTimestamp1 = eulerWithdrawsDb.iat[withdrawTicker,3]
        recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
        now = (int(dt.datetime.utcnow().timestamp()))
        datediff=abs(int(now) - recordTimestamp1)
        daysAgo = int(datediff/86400)
        eulerwithdrawsDbLength = len(eulerWithdrawsDb)
        eulerwithdrawListLength = len(withdrawslist)
        recordID = eulerWithdrawsDb.iat[withdrawTicker,0]
        withdraw = eulerWithdrawsDb.iat[withdrawTicker,1]
        print("iterations: ", withdrawTicker, "Lines skipped: ",skipValue, "records collected: ", eulerwithdrawListLength, " - latest recordID: ",withdraw, " ", recordID , " - withdraws DB length: ", eulerwithdrawsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
        #print("iterations: ", withdrawTicker, "records collected: ", withdrawListLength, " - userdb length: ", withdrawsDbLength)
        withdrawTicker = withdrawTicker+1

        if daysAgo>daysLimit: exit =True
        if withdrawListLength<1000: exit =True

    eulerWithdrawsDb.columns = eulerWithdrawsDb.columns.str.replace("withdraws", "transaction")
    eulerWithdrawsDb['event']='withdraw'
    eulerWithdrawsDb.drop_duplicates
    print("Done. Total Records collected: ", eulerwithdrawsDbLength)
except:
    eulerWithdrawsDb =pd.DataFrame()
    eulerWithdrawsDb['withdraws_id']=[]
    eulerWithdrawsDb['withdraws_hash']=[]
    eulerWithdrawsDb['withdraws_logIndex']=[]
    eulerWithdrawsDb['withdraws_timestamp']=[]
    eulerWithdrawsDb['withdraws_to']=[]
    eulerWithdrawsDb['withdraws_asset_id']=[]
    eulerWithdrawsDb['withdraws_asset_symbol']=[]
    eulerWithdrawsDb['withdraws_asset_decimals']=[]
    eulerWithdrawsDb['withdraws_asset_lastPriceUSD']=[]
    eulerWithdrawsDb['withdraws_asset_lastPriceBlockNumber']=[]
    eulerWithdrawsDb['withdraws_amount']=[]
    eulerWithdrawsDb['withdraws_amountUSD']=[]
    eulerWithdrawsDb['withdraws_from']=[]
eulerWithdrawsDb.head(10)

,withdraws_id,withdraws_hash,withdraws_logIndex,withdraws_timestamp,withdraws_to,withdraws_asset_id,withdraws_asset_symbol,withdraws_asset_decimals,withdraws_asset_lastPriceUSD,withdraws_asset_lastPriceBlockNumber,withdraws_amount,withdraws_amountUSD,withdraws_from


In [47]:
eulerWithdrawEventsDbTimedClean = pd.DataFrame()
eulerWithdrawsDb['transactionReadableAmount'] = eulerWithdrawsDb['withdraws_amount']/(10**(eulerWithdrawsDb['withdraws_asset_decimals']))

eulerWithdrawsDb['transactionReadableAmount']=eulerWithdrawsDb['transactionReadableAmount'].astype('float')
eulerWithdrawsDb['dateTime'] =(pd.to_datetime(eulerWithdrawsDb['withdraws_timestamp'],unit='s'))
eulerWithdrawsDb.head(10)

,withdraws_id,withdraws_hash,withdraws_logIndex,withdraws_timestamp,withdraws_to,withdraws_asset_id,withdraws_asset_symbol,withdraws_asset_decimals,withdraws_asset_lastPriceUSD,withdraws_asset_lastPriceBlockNumber,withdraws_amount,withdraws_amountUSD,withdraws_from,transactionReadableAmount,dateTime


In [48]:
try:
    eulerWithdrawEventsDbTimedClean = eulerWithdrawsDb['withdraws_id','withdraws_hash','withdraws_logIndex','withdraws_to','withdraws_asset_id','withdraws_asset_symbol','withdraws_asset_decimals', 'withdraws_asset_lastPriceUSD','withdraws_asset_lastPriceBlockNumber','withdraws_amount','withdraws_from','dateTime'].copy()
    eulerWithdrawEventsDbTimedClean.sort_values(by=['withdraws_id','withdraws_hash','withdraws_logIndex','withdraws_to','withdraws_asset_id','withdraws_asset_symbol','withdraws_asset_decimals', 'withdraws_asset_lastPriceUSD','withdraws_asset_lastPriceBlockNumber','withdraws_amount','withdraws_from','dateTime'])
    eulerWithdrawEventsDbTimedClean.drop_duplicates(subset=['withdraws_hash'],keep='first', inplace=True)
    eulerWithdrawEventsDbTimedClean['calDate'] = eulerWithdrawEventsDbTimedClean['dateTime'].dt.normalize()
    eulerWithdrawsDb.head(100)
except:
    print('Error.No events, perhaps?')

Error.No events, perhaps?


In [ ]:
****************************************************************************************************************
****************************************************************************************************************
COMBINE DEPOSIT EVENTS

****************************************************************************************************************
****************************************************************************************************************
COMBINE DEPOSIT EVENTS

In [49]:
#COLLATERALDEPOSITS
eulerDepositsDbPreMerge = db.query(
    "select distinct"
    " transaction_hash transaction_id "
    ", transaction_timestamp  "
    ", dateTime "
    ", transaction_from  "
    ", transaction_asset_symbol asset_symbol"
    ", transactionReadableAmount readableAmount "
    ", transaction_amountUSD transaction_amountUSD "
    "from eulerDepositsDb "
).df()
eulerDepositsDbPreMerge['protocol'] = 'Euler'
eulerDepositsDbPreMerge['action'] = 'depositCollateral'
eulerDepositsDbPreMerge.head(10)



,transaction_id,transaction_timestamp,dateTime,transaction_from,asset_symbol,readableAmount,transaction_amountUSD,protocol,action
0,0xffd4f953670dec6f8ae7d0d79391572912b056b99f6c...,1677139223,2023-02-23 08:00:23,0x642c8ed496a3d05a0ed08911c01f652a0c4b84d4,OHM,450.000000,4781.601428,Euler,depositCollateral
1,0x301069fd24dd70d15cdcbebd4f076efe486a82f34724...,1676615723,2023-02-17 06:35:23,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,OHM,4.973204,52.678831,Euler,depositCollateral
2,0xa7495eba745bd67279969c1b8687f816e0d83a60bf0c...,1676379695,2023-02-14 13:01:35,0x245cc372c84b3645bf0ffe6538620b04a217988b,OHM,30000.000000,310089.501811,Euler,depositCollateral


In [50]:
siloDepositEventsDbTimedClean.head(1)

,event,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_asset_name,transaction_asset_symbol,transactionReadableAmount,transaction_asset_lastPriceUSD,transaction_amountUSD,dateTime,calDate
12,deposit,0x1c37394aaabc2445a66b61a360cf08ae096b7582e2ed...,16646360,1676611691,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,Olympus,OHM,10.0,10.379649,105.92534,2023-02-17 05:28:11,2023-02-17


In [51]:
siloDepositsDbPreMerge = db.query(
    "select distinct"
    " transaction_hash  transaction_id"
    ", transaction_timestamp  "
    ", dateTime "
    ", transaction_account_id  transaction_from"
    ", transaction_asset_symbol asset_symbol "
    ", transactionReadableAmount readableAmount "
    ", transaction_amountUSD  "
    "from siloDepositEventsDbTimedClean "
).df()
siloDepositsDbPreMerge['protocol'] = 'Silo'
siloDepositsDbPreMerge['action'] = 'depositCollateral'
siloDepositsDbPreMerge.head(10)


,transaction_id,transaction_timestamp,dateTime,transaction_from,asset_symbol,readableAmount,transaction_amountUSD,protocol,action
0,0x1c37394aaabc2445a66b61a360cf08ae096b7582e2ed...,1676611691,2023-02-17 05:28:11,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,OHM,10.000000,105.925340,Silo,depositCollateral
1,0xf9bbcc923182fb6406e97fce0f92c22c87a284d55812...,1676379599,2023-02-14 12:59:59,0x245cc372c84b3645bf0ffe6538620b04a217988b,OHM,20000.000000,206726.334541,Silo,depositCollateral
2,0x0aa351eb8ff1ca2bd6c4e65229f83c7e5b596d72c889...,1676360171,2023-02-14 07:36:11,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,2.514698,26.317329,Silo,depositCollateral


In [52]:
ohmDeposits = pd.concat([eulerDepositsDbPreMerge,siloDepositsDbPreMerge])
ohmDeposits.head(100)

,transaction_id,transaction_timestamp,dateTime,transaction_from,asset_symbol,readableAmount,transaction_amountUSD,protocol,action
0,0xffd4f953670dec6f8ae7d0d79391572912b056b99f6c...,1677139223,2023-02-23 08:00:23,0x642c8ed496a3d05a0ed08911c01f652a0c4b84d4,OHM,450.000000,4781.601428,Euler,depositCollateral
1,0x301069fd24dd70d15cdcbebd4f076efe486a82f34724...,1676615723,2023-02-17 06:35:23,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,OHM,4.973204,52.678831,Euler,depositCollateral
2,0xa7495eba745bd67279969c1b8687f816e0d83a60bf0c...,1676379695,2023-02-14 13:01:35,0x245cc372c84b3645bf0ffe6538620b04a217988b,OHM,30000.000000,310089.501811,Euler,depositCollateral
0,0x1c37394aaabc2445a66b61a360cf08ae096b7582e2ed...,1676611691,2023-02-17 05:28:11,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,OHM,10.000000,105.925340,Silo,depositCollateral
1,0xf9bbcc923182fb6406e97fce0f92c22c87a284d55812...,1676379599,2023-02-14 12:59:59,0x245cc372c84b3645bf0ffe6538620b04a217988b,OHM,20000.000000,206726.334541,Silo,depositCollateral
2,0x0aa351eb8ff1ca2bd6c4e65229f83c7e5b596d72c889...,1676360171,2023-02-14 07:36:11,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,2.514698,26.317329,Silo,depositCollateral


In [53]:
ohmDeposits.columns = ohmDeposits.columns.str.replace("actions", "transaction")

In [54]:
ohmDeposits.head(100)

,transaction_id,transaction_timestamp,dateTime,transaction_from,asset_symbol,readableAmount,transaction_amountUSD,protocol,action
0,0xffd4f953670dec6f8ae7d0d79391572912b056b99f6c...,1677139223,2023-02-23 08:00:23,0x642c8ed496a3d05a0ed08911c01f652a0c4b84d4,OHM,450.000000,4781.601428,Euler,depositCollateral
1,0x301069fd24dd70d15cdcbebd4f076efe486a82f34724...,1676615723,2023-02-17 06:35:23,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,OHM,4.973204,52.678831,Euler,depositCollateral
2,0xa7495eba745bd67279969c1b8687f816e0d83a60bf0c...,1676379695,2023-02-14 13:01:35,0x245cc372c84b3645bf0ffe6538620b04a217988b,OHM,30000.000000,310089.501811,Euler,depositCollateral
0,0x1c37394aaabc2445a66b61a360cf08ae096b7582e2ed...,1676611691,2023-02-17 05:28:11,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,OHM,10.000000,105.925340,Silo,depositCollateral
1,0xf9bbcc923182fb6406e97fce0f92c22c87a284d55812...,1676379599,2023-02-14 12:59:59,0x245cc372c84b3645bf0ffe6538620b04a217988b,OHM,20000.000000,206726.334541,Silo,depositCollateral
2,0x0aa351eb8ff1ca2bd6c4e65229f83c7e5b596d72c889...,1676360171,2023-02-14 07:36:11,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,2.514698,26.317329,Silo,depositCollateral


In [55]:
path =file+'/allOhmDeposits.csv'
ohmDeposits.to_csv(path, index = False)

path =file+'/allOhmDeposits_piped.csv'
ohmDeposits.to_csv(path, index = False, sep='|')
print(path, 'file saved,', len(ohmDeposits),' records')

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\ohmBorrowing/data/allOhmDeposits_piped.csv file saved, 6  records


****************************************************************************************************************
****************************************************************************************************************
COMBINE WITHDRAWAL EVENTS

In [56]:
siloWithdrawsDb['protocol'] = 'Silo'
siloWithdrawsDb.head(1)

,transaction_id,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_market_protocol_id,transaction_market_rates_rate,transaction_market_rates_duration,transaction_market_rates_side,...,transaction_asset_name,transaction_asset_symbol,transaction_asset_decimals,transaction_asset_lastPriceUSD,transaction_amount,transaction_amountUSD,event,transactionReadableAmount,dateTime,protocol
0,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,16627397,1676382659,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,5.656345,None,BORROWER,...,Olympus,OHM,9,10.379649,2514698283,25.992718,withdraw,2.514698,2023-02-14 13:50:59,Silo


In [57]:
allCollateralWithdrawals = pd.concat([siloWithdrawsDb])
allCollateralWithdrawals.head(10)

,transaction_id,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_market_protocol_id,transaction_market_rates_rate,transaction_market_rates_duration,transaction_market_rates_side,...,transaction_asset_name,transaction_asset_symbol,transaction_asset_decimals,transaction_asset_lastPriceUSD,transaction_amount,transaction_amountUSD,event,transactionReadableAmount,dateTime,protocol
0,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,16627397,1676382659,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,5.656345,None,BORROWER,...,Olympus,OHM,9,10.379649,2514698283,25.992718,withdraw,2.514698,2023-02-14 13:50:59,Silo
1,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,16627397,1676382659,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,...,Olympus,OHM,9,10.379649,2514698283,25.992718,withdraw,2.514698,2023-02-14 13:50:59,Silo
2,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,16627397,1676382659,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,...,Olympus,OHM,9,10.379649,2514698283,25.992718,withdraw,2.514698,2023-02-14 13:50:59,Silo


****************************************************************************************************************
****************************************************************************************************************
COMBINE WITHDRAWAL AND DEPOSIT EVENTS

In [58]:
ohmDeposits.columns = ohmDeposits.columns.str.replace("deposit", "transaction")
ohmDeposits.head(10)

,transaction_id,transaction_timestamp,dateTime,transaction_from,asset_symbol,readableAmount,transaction_amountUSD,protocol,action
0,0xffd4f953670dec6f8ae7d0d79391572912b056b99f6c...,1677139223,2023-02-23 08:00:23,0x642c8ed496a3d05a0ed08911c01f652a0c4b84d4,OHM,450.000000,4781.601428,Euler,depositCollateral
1,0x301069fd24dd70d15cdcbebd4f076efe486a82f34724...,1676615723,2023-02-17 06:35:23,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,OHM,4.973204,52.678831,Euler,depositCollateral
2,0xa7495eba745bd67279969c1b8687f816e0d83a60bf0c...,1676379695,2023-02-14 13:01:35,0x245cc372c84b3645bf0ffe6538620b04a217988b,OHM,30000.000000,310089.501811,Euler,depositCollateral
0,0x1c37394aaabc2445a66b61a360cf08ae096b7582e2ed...,1676611691,2023-02-17 05:28:11,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,OHM,10.000000,105.925340,Silo,depositCollateral
1,0xf9bbcc923182fb6406e97fce0f92c22c87a284d55812...,1676379599,2023-02-14 12:59:59,0x245cc372c84b3645bf0ffe6538620b04a217988b,OHM,20000.000000,206726.334541,Silo,depositCollateral
2,0x0aa351eb8ff1ca2bd6c4e65229f83c7e5b596d72c889...,1676360171,2023-02-14 07:36:11,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,2.514698,26.317329,Silo,depositCollateral


In [59]:
allCollateralWithdrawals['dateTime'] =(pd.to_datetime(allCollateralWithdrawals['transaction_timestamp'],unit='s'))
allCollateralWithdrawals.head(10)

,transaction_id,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_market_protocol_id,transaction_market_rates_rate,transaction_market_rates_duration,transaction_market_rates_side,...,transaction_asset_name,transaction_asset_symbol,transaction_asset_decimals,transaction_asset_lastPriceUSD,transaction_amount,transaction_amountUSD,event,transactionReadableAmount,dateTime,protocol
0,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,16627397,1676382659,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,5.656345,None,BORROWER,...,Olympus,OHM,9,10.379649,2514698283,25.992718,withdraw,2.514698,2023-02-14 13:50:59,Silo
1,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,16627397,1676382659,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,...,Olympus,OHM,9,10.379649,2514698283,25.992718,withdraw,2.514698,2023-02-14 13:50:59,Silo
2,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,16627397,1676382659,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,...,Olympus,OHM,9,10.379649,2514698283,25.992718,withdraw,2.514698,2023-02-14 13:50:59,Silo


In [60]:
allCollateralWithdrawals = db.query("select "
                                    " transaction_hash transaction_id"
                                    ",transaction_timestamp "
                                    ", dateTime "
                                    ", transaction_account_id  transaction_from "
                                    ", transaction_asset_symbol asset_symbol"
                                    ", transactionReadableAmount readableAmount "
                                    ", transaction_amountUSD "
                                    ", protocol "
                                    ", 'withdrawCollateral' as action "
                                    "from allCollateralWithdrawals "
                                    "").df()
allCollateralWithdrawals.head(10)

,transaction_id,transaction_timestamp,dateTime,transaction_from,asset_symbol,readableAmount,transaction_amountUSD,protocol,action
0,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,1676382659,2023-02-14 13:50:59,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,2.514698,25.992718,Silo,withdrawCollateral
1,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,1676382659,2023-02-14 13:50:59,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,2.514698,25.992718,Silo,withdrawCollateral
2,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,1676382659,2023-02-14 13:50:59,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,2.514698,25.992718,Silo,withdrawCollateral


In [61]:
allOhmBorrowMarketEvents = pd.concat([ohmDeposits,allCollateralWithdrawals])
allOhmBorrowMarketEvents['dateTime'] =(pd.to_datetime(allOhmBorrowMarketEvents['transaction_timestamp'],unit='s'))
allOhmBorrowMarketEvents.head(100)

,transaction_id,transaction_timestamp,dateTime,transaction_from,asset_symbol,readableAmount,transaction_amountUSD,protocol,action
0,0xffd4f953670dec6f8ae7d0d79391572912b056b99f6c...,1677139223,2023-02-23 08:00:23,0x642c8ed496a3d05a0ed08911c01f652a0c4b84d4,OHM,450.000000,4781.601428,Euler,depositCollateral
1,0x301069fd24dd70d15cdcbebd4f076efe486a82f34724...,1676615723,2023-02-17 06:35:23,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,OHM,4.973204,52.678831,Euler,depositCollateral
2,0xa7495eba745bd67279969c1b8687f816e0d83a60bf0c...,1676379695,2023-02-14 13:01:35,0x245cc372c84b3645bf0ffe6538620b04a217988b,OHM,30000.000000,310089.501811,Euler,depositCollateral
0,0x1c37394aaabc2445a66b61a360cf08ae096b7582e2ed...,1676611691,2023-02-17 05:28:11,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,OHM,10.000000,105.925340,Silo,depositCollateral
1,0xf9bbcc923182fb6406e97fce0f92c22c87a284d55812...,1676379599,2023-02-14 12:59:59,0x245cc372c84b3645bf0ffe6538620b04a217988b,OHM,20000.000000,206726.334541,Silo,depositCollateral
2,0x0aa351eb8ff1ca2bd6c4e65229f83c7e5b596d72c889...,1676360171,2023-02-14 07:36:11,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,2.514698,26.317329,Silo,depositCollateral
0,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,1676382659,2023-02-14 13:50:59,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,2.514698,25.992718,Silo,withdrawCollateral
1,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,1676382659,2023-02-14 13:50:59,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,2.514698,25.992718,Silo,withdrawCollateral
2,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,1676382659,2023-02-14 13:50:59,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,2.514698,25.992718,Silo,withdrawCollateral


****************************************************************************************************************
****************************************************************************************************************
COMBINE BORROW EVENTS

In [62]:
siloBorrowEventsDbTimedClean.head(200)

,event,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_asset_name,transaction_asset_symbol,transactionReadableAmount,transaction_asset_lastPriceUSD,transaction_amountUSD,dateTime,calDate
0,borrow,0x3063cfbced7d9431862437e31adcc3581e9df1cd87d3...,16674201,1676950007,0xf659faac0401cc19110321dcc7989c101cced8e4,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,Olympus,OHM,6420.000000,10.379649,69617.631237,2023-02-21 03:26:47,2023-02-21
6,borrow,0x264ecb6cc9089ebf8fcb6f8ae5ce26358b15b0551e61...,16648400,1676636519,0xcb6e1613029d790c00f89296808f278d6dc25b2f,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,Olympus,OHM,58.000000,10.379649,615.628222,2023-02-17 12:21:59,2023-02-17
12,borrow,0x6726ea9fa581ab0a5fa26b8a112c95e690500f446c39...,16628130,1676391467,0x9b8b04b6f82cd5e1dae58ca3614d445f93defc5c,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,Olympus,OHM,1480.000000,10.379649,15782.938441,2023-02-14 16:17:47,2023-02-14
18,borrow,0x5fca09aaf64619a5d644ed392a64f5bd08b9e6b4b46e...,16627404,1676382743,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db,Olympus,OHM,175.320428,10.379649,1812.167469,2023-02-14 13:52:23,2023-02-14


In [63]:
siloBorrowsDbPreMerge = db.query(
    "select distinct"
    " transaction_hash transaction_id "
    ", transaction_timestamp transaction_timestamp "
    ", dateTime "
    ", transaction_account_id transaction_from "
    ", transaction_asset_symbol  asset_symbol"
    ", transactionReadableAmount readableAmount "
    ", transaction_amountUSD transaction_amountUSD "
    "from siloBorrowEventsDbTimedClean "
).df()
siloBorrowsDbPreMerge['action'] = 'borrowAsset'
siloBorrowsDbPreMerge['protocol'] = 'Silo'
siloBorrowsDbPreMerge.head(10)


,transaction_id,transaction_timestamp,dateTime,transaction_from,asset_symbol,readableAmount,transaction_amountUSD,action,protocol
0,0x3063cfbced7d9431862437e31adcc3581e9df1cd87d3...,1676950007,2023-02-21 03:26:47,0xf659faac0401cc19110321dcc7989c101cced8e4,OHM,6420.000000,69617.631237,borrowAsset,Silo
1,0x264ecb6cc9089ebf8fcb6f8ae5ce26358b15b0551e61...,1676636519,2023-02-17 12:21:59,0xcb6e1613029d790c00f89296808f278d6dc25b2f,OHM,58.000000,615.628222,borrowAsset,Silo
2,0x6726ea9fa581ab0a5fa26b8a112c95e690500f446c39...,1676391467,2023-02-14 16:17:47,0x9b8b04b6f82cd5e1dae58ca3614d445f93defc5c,OHM,1480.000000,15782.938441,borrowAsset,Silo
3,0x5fca09aaf64619a5d644ed392a64f5bd08b9e6b4b46e...,1676382743,2023-02-14 13:52:23,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,175.320428,1812.167469,borrowAsset,Silo


EVENTUALLY THERE WILL BE EULER BORROWS

In [64]:
allOhmBorrowEvents = pd.concat([siloBorrowsDbPreMerge])
allOhmBorrowEvents.head(10)

,transaction_id,transaction_timestamp,dateTime,transaction_from,asset_symbol,readableAmount,transaction_amountUSD,action,protocol
0,0x3063cfbced7d9431862437e31adcc3581e9df1cd87d3...,1676950007,2023-02-21 03:26:47,0xf659faac0401cc19110321dcc7989c101cced8e4,OHM,6420.000000,69617.631237,borrowAsset,Silo
1,0x264ecb6cc9089ebf8fcb6f8ae5ce26358b15b0551e61...,1676636519,2023-02-17 12:21:59,0xcb6e1613029d790c00f89296808f278d6dc25b2f,OHM,58.000000,615.628222,borrowAsset,Silo
2,0x6726ea9fa581ab0a5fa26b8a112c95e690500f446c39...,1676391467,2023-02-14 16:17:47,0x9b8b04b6f82cd5e1dae58ca3614d445f93defc5c,OHM,1480.000000,15782.938441,borrowAsset,Silo
3,0x5fca09aaf64619a5d644ed392a64f5bd08b9e6b4b46e...,1676382743,2023-02-14 13:52:23,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,175.320428,1812.167469,borrowAsset,Silo


In [65]:
allCollateralWithdrawals.head(10)

,transaction_id,transaction_timestamp,dateTime,transaction_from,asset_symbol,readableAmount,transaction_amountUSD,protocol,action
0,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,1676382659,2023-02-14 13:50:59,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,2.514698,25.992718,Silo,withdrawCollateral
1,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,1676382659,2023-02-14 13:50:59,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,2.514698,25.992718,Silo,withdrawCollateral
2,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,1676382659,2023-02-14 13:50:59,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,2.514698,25.992718,Silo,withdrawCollateral


In [66]:
ohmDeposits.head(10)

,transaction_id,transaction_timestamp,dateTime,transaction_from,asset_symbol,readableAmount,transaction_amountUSD,protocol,action
0,0xffd4f953670dec6f8ae7d0d79391572912b056b99f6c...,1677139223,2023-02-23 08:00:23,0x642c8ed496a3d05a0ed08911c01f652a0c4b84d4,OHM,450.000000,4781.601428,Euler,depositCollateral
1,0x301069fd24dd70d15cdcbebd4f076efe486a82f34724...,1676615723,2023-02-17 06:35:23,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,OHM,4.973204,52.678831,Euler,depositCollateral
2,0xa7495eba745bd67279969c1b8687f816e0d83a60bf0c...,1676379695,2023-02-14 13:01:35,0x245cc372c84b3645bf0ffe6538620b04a217988b,OHM,30000.000000,310089.501811,Euler,depositCollateral
0,0x1c37394aaabc2445a66b61a360cf08ae096b7582e2ed...,1676611691,2023-02-17 05:28:11,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,OHM,10.000000,105.925340,Silo,depositCollateral
1,0xf9bbcc923182fb6406e97fce0f92c22c87a284d55812...,1676379599,2023-02-14 12:59:59,0x245cc372c84b3645bf0ffe6538620b04a217988b,OHM,20000.000000,206726.334541,Silo,depositCollateral
2,0x0aa351eb8ff1ca2bd6c4e65229f83c7e5b596d72c889...,1676360171,2023-02-14 07:36:11,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,2.514698,26.317329,Silo,depositCollateral


In [67]:
allOhmBorrowMarketEvents = pd.DataFrame()
allOhmBorrowMarketEvents = pd.concat([ohmDeposits,allCollateralWithdrawals,allOhmBorrowEvents])
allOhmBorrowMarketEvents['dateTime'] =(pd.to_datetime(allOhmBorrowMarketEvents['transaction_timestamp'],unit='s'))
allOhmBorrowMarketEvents.head(1000)

,transaction_id,transaction_timestamp,dateTime,transaction_from,asset_symbol,readableAmount,transaction_amountUSD,protocol,action
0,0xffd4f953670dec6f8ae7d0d79391572912b056b99f6c...,1677139223,2023-02-23 08:00:23,0x642c8ed496a3d05a0ed08911c01f652a0c4b84d4,OHM,450.000000,4781.601428,Euler,depositCollateral
1,0x301069fd24dd70d15cdcbebd4f076efe486a82f34724...,1676615723,2023-02-17 06:35:23,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,OHM,4.973204,52.678831,Euler,depositCollateral
2,0xa7495eba745bd67279969c1b8687f816e0d83a60bf0c...,1676379695,2023-02-14 13:01:35,0x245cc372c84b3645bf0ffe6538620b04a217988b,OHM,30000.000000,310089.501811,Euler,depositCollateral
0,0x1c37394aaabc2445a66b61a360cf08ae096b7582e2ed...,1676611691,2023-02-17 05:28:11,0x9b53bed0d62028c0fc5d2006c076cb33e6e6cecd,OHM,10.000000,105.925340,Silo,depositCollateral
1,0xf9bbcc923182fb6406e97fce0f92c22c87a284d55812...,1676379599,2023-02-14 12:59:59,0x245cc372c84b3645bf0ffe6538620b04a217988b,OHM,20000.000000,206726.334541,Silo,depositCollateral
2,0x0aa351eb8ff1ca2bd6c4e65229f83c7e5b596d72c889...,1676360171,2023-02-14 07:36:11,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,2.514698,26.317329,Silo,depositCollateral
0,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,1676382659,2023-02-14 13:50:59,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,2.514698,25.992718,Silo,withdrawCollateral
1,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,1676382659,2023-02-14 13:50:59,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,2.514698,25.992718,Silo,withdrawCollateral
2,0xa1821732de0a64b8c5f1b4371e115d1a116bf143c8c7...,1676382659,2023-02-14 13:50:59,0xc0b1f200aa9546ce074397bed7438bb2ec033877,OHM,2.514698,25.992718,Silo,withdrawCollateral
0,0x3063cfbced7d9431862437e31adcc3581e9df1cd87d3...,1676950007,2023-02-21 03:26:47,0xf659faac0401cc19110321dcc7989c101cced8e4,OHM,6420.000000,69617.631237,Silo,borrowAsset


In [68]:
path =file+'/allOhmBorrowMarketEvents.csv'
allOhmBorrowMarketEvents.to_csv(path, index = False)

path =file+'/allOhmBorrowMarketEvents_piped.csv'
allOhmBorrowMarketEvents.to_csv(path, index = False, sep='|')
print(path, 'file saved,', len(ohmDeposits),' records')

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\ohmBorrowing/data/allOhmBorrowMarketEvents_piped.csv file saved, 6  records


In [69]:
ohmBorrowMarketLevels = pd.DataFrame({
    'protocol':['Silo','Euler'],
    'collateralBalance':[collateralBalance,totalDepositedEuler],
    'totalBorrowedOhm':[totalBorrowedOHM,0]
})
ohmBorrowMarketLevels.head(10)

,protocol,collateralBalance,totalBorrowedOhm
0,Silo,20010.000000,8133.320428
1,Euler,30454.973204,0.000000


In [70]:
path =file+'/ohmBorrowMarketLevels.csv'
ohmBorrowMarketLevels.to_csv(path, index = False)

path =file+'/ohmBorrowMarketLevels_piped.csv'
ohmBorrowMarketLevels.to_csv(path, index = False, sep='|')
print(path, 'file saved,', len(ohmBorrowMarketLevels),' records')

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\ohmBorrowing/data/ohmBorrowMarketLevels_piped.csv file saved, 2  records
